# Kùzu Graph Store

This notebook walks through configuring `Kùzu` to be the backend for graph storage in LlamaIndex.

In [3]:
# My OpenAI Key
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare for Kùzu

In [5]:
# Clean up all the directories used in this notebook
import shutil

shutil.rmtree("./test1", ignore_errors=True)
shutil.rmtree("./test2", ignore_errors=True)
shutil.rmtree("./test3", ignore_errors=True)

In [6]:
%pip install kuzu
import kuzu

db = kuzu.Database("test1")

     |████████████████████████████████| 5.5 MB 4.8 MB/s eta 0:00:01
You should consider upgrading via the '/Users/loganmarkewich/llama_index/llama-index/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Using Knowledge Graph with KuzuGraphStore

In [7]:
from llama_index.graph_stores import KuzuGraphStore

graph_store = KuzuGraphStore(db)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


#### Building the Knowledge Graph

In [8]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    KnowledgeGraphIndex,
)

from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import kuzu

In [9]:
documents = SimpleDirectoryReader(
    "../../../../examples/paul_graham_essay/data"
).load_data()

In [10]:
# define LLM

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [11]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

#### Querying the Knowledge Graph

First, we can query and send only the triplets to the LLM.

In [12]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about Interleaf


INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge sequence in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made', 'software for creating documents']
Interleaf ['added', 'scripting language']
Interleaf ['taught', 'what not to do']


In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>Interleaf is a company that made software for creating documents. They also added a scripting language to their software. Additionally, they taught what not to do.</b>

For more detailed answers, we can also send the text from where the retrieved tripets were extracted.

In [14]:
query_engine = index.as_query_engine(include_text=True, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 144f784c-d052-4fed-86f8-c895da6e13df: each student had. But the Accademia wasn't teaching me anything except Italia...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 7c877dd3-3375-4ab7-8745-e0dfbabfe5bd: learned some useful things at Interleaf, though they were mostly about what n...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge sequence in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made', 'software for creating documents']
Interleaf ['added', 'scripting language']
In

In [15]:
display(Markdown(f"<b>{response}</b>"))

<b>Interleaf was a company that made software for creating documents. They were inspired by Emacs and added a scripting language to their software, which was a dialect of Lisp. The company hired a Lisp hacker to write things in this scripting language. The narrator worked at Interleaf for a year but admits to being a bad employee. They found it difficult to understand most of the software because it was primarily written in C, a language they did not know or want to learn. Despite this, they were paid well and managed to save enough money to go back to RISD and pay off their college loans. The narrator also learned some valuable lessons at Interleaf, such as the importance of having product people rather than sales people running technology companies, the drawbacks of having too many people edit code, the impact of office space on productivity, the value of corridor conversations over planned meetings, the challenges of dealing with big bureaucratic customers, and the importance of being the "entry level" option in a market.</b>

#### Query with embeddings

In [16]:
# NOTE: can take a while!
db = kuzu.Database("test2")
graph_store = KuzuGraphStore(db)
storage_context = StorageContext.from_defaults(graph_store=graph_store)
new_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    service_context=service_context,
    storage_context=storage_context,
    include_embeddings=True,
)

In [17]:
rel_map = graph_store.get_rel_map()

In [18]:
# query using top 3 triplets plus keywords (duplicate triplets are removed)
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about what the author worked on at Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf', 'author', 'worked']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 144f784c-d052-4fed-86f8-c895da6e13df: each student had. But the Accademia wasn't teaching me anything except Italia...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 7c877dd3-3375-4ab7-8745-e0dfbabfe5bd: learned some useful things at Interleaf, though they were mostly about what n...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge sequence in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made', 'software for creating documen

In [19]:
display(Markdown(f"<b>{response}</b>"))

<b>At Interleaf, the author worked on creating software for creating documents. They also worked on adding a scripting language, which was inspired by Emacs and was a dialect of Lisp. However, the author admits to being a bad employee and not fully understanding the software, as it was primarily written in C. They also mention that they spent a lot of time working on their book "On Lisp" during their time at Interleaf. Overall, the author learned some useful things at Interleaf, particularly about what not to do in technology companies.</b>

#### Visualizing the Graph

In [20]:
%pip install pyvis

     |████████████████████████████████| 756 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 4.1 MB/s eta 0:00:01
You should consider upgrading via the '/Users/loganmarkewich/llama_index/llama-index/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
## create graph
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("kuzugraph_draw.html")

kuzugraph_draw.html


#### [Optional] Try building the graph and manually add triplets!

In [23]:
from llama_index.node_parser import SimpleNodeParser

In [24]:
node_parser = SimpleNodeParser()

In [25]:
nodes = node_parser.get_nodes_from_documents(documents)

In [26]:
# initialize an empty database
db = kuzu.Database("test3")
graph_store = KuzuGraphStore(db)
storage_context = StorageContext.from_defaults(graph_store=graph_store)
index = KnowledgeGraphIndex(
    [],
    service_context=service_context,
    storage_context=storage_context,
)

In [27]:
# add keyword mappings and nodes manually
# add triplets (subject, relationship, object)

# for node 0
node_0_tups = [
    ("author", "worked on", "writing"),
    ("author", "worked on", "programming"),
]
for tup in node_0_tups:
    index.upsert_triplet_and_node(tup, nodes[0])

# for node 1
node_1_tups = [
    ("Interleaf", "made software for", "creating documents"),
    ("Interleaf", "added", "scripting language"),
    ("software", "generate", "web sites"),
]
for tup in node_1_tups:
    index.upsert_triplet_and_node(tup, nodes[1])

In [28]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge sequence in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made software for', 'creating documents']
Interleaf ['added', 'scripting language']


In [29]:
str(response)

'Interleaf is a software company that specializes in creating documents. They have also added a scripting language to their software.'